In [1]:
%pip install --upgrade llama-index        # core
%pip install --upgrade llama-index-llms-google-genai  # Google / Gemini LLM integration
%pip install --upgrade llama-index-embeddings-google-genai  # embeddings via Google GenAI
%pip install --upgrade google-generativeai  # underlying Google SDK
%pip install --upgrade pinecone-client  


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
  Using cached pinecone_client-6.0.0-py3-none-any.whl.metadata (3.4 kB)
Using cached pinecone_client-6.0.0-py3-none-any.whl (6.7 kB)

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated pa

In [2]:
import os
from llama_index.core import (
 SimpleDirectoryReader,
 Settings,
 SummaryIndex,
 TreeIndex,
 KeywordTableIndex, # <-- correct import (core)
)
from llama_index.llms.google_genai import GoogleGenAI
from llama_index.embeddings.google_genai import GoogleGenAIEmbedding

In [4]:
# ============================ Embedding / LLM layer
#============================
os.environ.pop("OPENAI_API_KEY", None) # avoid fallback
Settings.embed_model = GoogleGenAIEmbedding(
 model_name="models/text-embedding-004",
 api_key="",
)
Settings.llm = GoogleGenAI(
 model="gemini-2.5-flash",
 api_key="",
)

In [9]:
print("CWD:", os.getcwd())

CWD: /Users/devenderswami


In [10]:
# ============================ Data layer (PDFs only)
#============================
# Read ONLY PDFs from ./coffee_docs
try:
 docs = SimpleDirectoryReader(
 input_dir="./GenAI/GenAI-NoteBooks/coffee_pages",
 required_exts=[".html"],
 ).load_data()
except TypeError:
 # fallback for older versions
 docs = SimpleDirectoryReader(
 input_dir="./GenAI/GenAI-NoteBooks/coffee_pages",
 file_exts=[".html"],
 ).load_data()


In [12]:
# ============================ Build separate indexes (no fusion)
# ============================
summary_idx = SummaryIndex.from_documents(docs) # precomputed per-doc summaries
keyword_idx = KeywordTableIndex.from_documents(docs) # exact/term lookups
tree_idx = TreeIndex.from_documents(docs) # hierarchical

RuntimeError: asyncio.run() cannot be called from a running event loop

In [14]:
import nest_asyncio
nest_asyncio.apply()

In [15]:
#============================ Query layer (each used independently)
#============================
# 1) SummaryIndex -- "explain the 200-page policy" style
summary_qe = summary_idx.as_query_engine(llm=Settings.llm)
print(summary_qe.query("Give a high-level summary of these PDFs in 5bullets."))
# 2) Keyword Table Index -- fast exact/term matches
keyword_qe = keyword_idx.as_query_engine(llm=Settings.llm)
print(keyword_qe.query("Find occurrences where 'ashwagandha' appears in titles or headings."))
# 3) Tree Index -- navigate top→down like a table of contents
tree_qe = tree_idx.as_query_engine(llm=Settings.llm)
print(tree_qe.query("Locate sections about brewing temperature and summarize them."))

2025-11-28 23:28:48,224 - INFO - AFC is enabled with max remote calls: 10.
2025-11-28 23:28:56,690 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash:generateContent "HTTP/1.1 200 OK"


*   The collection details various Indian-inspired coffee preparations, frequently incorporating traditional spices and herbs such as cardamom, ginger, turmeric, ashwagandha, tulsi, pepper, and saffron for distinct flavor profiles.
*   Specific Indian coffee styles are highlighted, including the frothy South Indian Filter Coffee, often blended with chicory, and Beaten Coffee, a home-style whipped instant coffee known for its creamy foam.
*   Alternative ingredients are explored, such as jaggery for a caramel-like sweetness and coconut milk for a creamy, dairy-free option, particularly in coastal-style beverages.
*   Modern wellness-focused blends are featured, combining coffee with adaptogenic ingredients like ashwagandha and various mushroom powders for a contemporary twist.
*   A conceptual overview is provided on approaching coffee consumption through an Ayurvedic lens, encouraging mindful practices regarding timing, quantity, and personalization.


NameError: name 'keyword_idx' is not defined